In [ ]:
import pandas as pd
import huggingface_hub
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    Trainer, 
    TrainingArguments,
    AutoConfig
)
from datasets import Dataset
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
df = pd.read_csv('../data/train_dataset.csv')
df_val = pd.read_csv('../data/valid_dataset.csv')

In [ ]:
def numerize(df):
    df['label'] = 0
    df.loc[df['genre'] == "giao-duc", "label"] = 1
    df.loc[df['genre'] == "xe", "label"] = 2
    df.loc[df['genre'] == "suc-khoe", "label"] = 3
    df.loc[df['genre'] == "cong-nghe-game", "label"] = 4

numerize(df)
numerize(df_val)

In [ ]:
df_train = Dataset.from_pandas(df)
df_val = Dataset.from_pandas(df_val)

dataset = {
    'train': df_train,
    'validation': df_val,
}

In [ ]:
# vinai/phobert-base
model_name = "vinai/phobert-base"

# tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, add_eos_token=True)

In [ ]:
def tokenize(examples):
    tokenized_inputs = tokenizer(examples['title'], padding=True, truncation=True )
    return tokenized_inputs

dataset['train'] = dataset['train'].map(tokenize, batched=True, batch_size=None)
dataset['validation'] = dataset['validation'].map(tokenize, batched=True, batch_size=None)

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [ ]:
labels = list(set(df["label"]))
id2label = {k:v for k,v in enumerate(labels)}
label2id = {v:k for k,v in enumerate(labels)}
num_labels = len(labels)

config = (AutoConfig
          .from_pretrained(model_name, num_labels=num_labels,
                           label2id=label2id, id2label=id2label))

# model
model = AutoModelForSequenceClassification.from_pretrained(model_name, config=config)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [9]:
training_args = TrainingArguments(
                    output_dir="./results",
                    num_train_epochs=10,
                    learning_rate=2e-5,
                    per_device_train_batch_size=16,
                    per_device_eval_batch_size=16,
                    weight_decay=0.01,
                    logging_steps=len(dataset["train"]) // 16,
                    evaluation_strategy='epoch',
                    eval_steps=10,
                    report_to='none'  
            )

trainer = Trainer(
                  model=model,
                  args=training_args,
                  compute_metrics=compute_metrics,
                  train_dataset=dataset["train"],
                  eval_dataset=dataset["validation"],
                  data_collator=data_collator,
                  tokenizer=tokenizer
            )
trainer.train()

 30%|██▉       | 186/630 [01:46<03:50,  1.93it/s]

{'loss': 0.1786, 'grad_norm': 10.731210708618164, 'learning_rate': 1.4095238095238097e-05, 'epoch': 2.95}


                                                 
 30%|███       | 189/630 [01:48<03:38,  2.02it/s]

{'eval_loss': 0.25506022572517395, 'eval_accuracy': 0.92, 'eval_f1': 0.9225972273340695, 'eval_runtime': 0.5718, 'eval_samples_per_second': 174.871, 'eval_steps_per_second': 12.241, 'epoch': 3.0}


 39%|███▉      | 248/630 [02:18<03:09,  2.01it/s]

...

 98%|█████████▊| 620/630 [05:31<00:05,  1.97it/s]

{'loss': 0.0098, 'grad_norm': 0.07069993764162064, 'learning_rate': 3.174603174603175e-07, 'epoch': 9.84}


                                                 
100%|██████████| 630/630 [05:36<00:00,  1.87it/s]

{'eval_loss': 0.374647855758667, 'eval_accuracy': 0.93, 'eval_f1': 0.9320572450805009, 'eval_runtime': 0.588, 'eval_samples_per_second': 170.074, 'eval_steps_per_second': 11.905, 'epoch': 10.0}
{'train_runtime': 336.7096, 'train_samples_per_second': 29.699, 'train_steps_per_second': 1.871, 'train_loss': 0.2042935320782283, 'epoch': 10.0}


TrainOutput(global_step=630, training_loss=0.2042935320782283, metrics={'train_runtime': 336.7096, 'train_samples_per_second': 29.699, 'train_steps_per_second': 1.871, 'train_loss': 0.2042935320782283, 'epoch': 10.0})

In [ ]:
import torch
import torch.nn.functional as F
from sklearn.metrics import f1_score, recall_score, accuracy_score, precision_score

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("mob2711/phoBERT_finetune_news_classification")
model = AutoModelForSequenceClassification.from_pretrained("mob2711/phoBERT_finetune_news_classification")

def tokenize(text):
    encoded_text = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    return encoded_text

test = pd.read_csv('../data/test_dataset.csv')

In [ ]:
numerize(test)

In [ ]:
def predict(text_data):
    encoded_data = tokenize(text_data)
    with torch.no_grad():
        outputs = model(**encoded_data)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
    return predictions.cpu().numpy()

def predict_prob(text_data):
    encoded_data = tokenize(text_data)
    with torch.no_grad():
        outputs = model(**encoded_data)
        logits = outputs.logits
        probabilities = F.softmax(logits, dim=-1)
        predictions = torch.argmax(logits, dim=-1)
    return predictions.cpu().numpy(), probabilities.cpu().numpy()

In [ ]:
predictions = predict(test)

f1 = f1_score(test['label'], predictions, average='weighted')

recall = recall_score(test['label'], predictions, average='weighted')

precision = precision_score(test['label'], predictions, average='weighted')

accuracy = accuracy_score(test['label'], predictions)

print('accuracy: ' + str(accuracy))
print('recall: ' + str(recall))
print('precision: ' + str(precision))
print('f1: ' + str(f1))